<a href="https://colab.research.google.com/github/thirayume/muangtai/blob/main/Muangtai_PGSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Muangtai PostgreSQL Database export query to CSV**

In [ ]:
# @title Mount Google Drive (if need)

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Define secrets

from google.colab import userdata
host = userdata.get('host')
port = userdata.get('port')
database = userdata.get('database')
user = userdata.get('user')
password = userdata.get('password')

In [ ]:
# @title Install dependencies (if need)

!pip install psycopg2 pandas

In [ ]:
# @title Import dependencies

import psycopg2

import numpy as np
import pandas as pd
from pandas import plotting

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected = True)

from sklearn.cluster import KMeans
import fastai

import warnings
import os
from pathlib import Path
warnings.filterwarnings("ignore")

In [ ]:
# @title Define data path

data_path = Path('/content/drive/MyDrive/Colab Notebooks/Muangtai')

%cd {data_path}
%ls

In [ ]:
# @title Connect to the PostgreSQL database

conn = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password,
)
conn

In [ ]:
# @title Define a SQL to create a "view_picking_list" (if not found)

sql = '''
DROP VIEW IF EXISTS view_fact_picking_lists;

CREATE OR REPLACE VIEW view_fact_picking_lists
AS

select
fact_picking_lists.id as fact_picking_lists__id
,dim_dates."year" || '-' || dim_dates."month" || '-' || dim_dates."date" as fact_picking_lists__date
,to_char((LPAD((dim_times.hour_of_day || ''), 2, '0') || '' || LPAD((dim_times.minute_of_hour || ''), 2, '0') || '00')::time,'HH24:MI:SS AM') as fact_picking_lists__time
,GREATEST(
    dim_picking_lists.created_at,
    dim_picking_lists.updated_at,
    dim_picking_lists.moderated_at, (
    dim_picking_lists.document_date + '08:30:01'::time)
) as lasted_updated_at
,dim_picking_lists.document_number as dim_picking_lists__document_number
,dim_picking_lists.id as dim_picking_lists__id
,dim_stock_keeping_units.id as dim_stock_keeping_units__sku_id
,fact_picking_lists.quantity as fact_picking_lists__quantity
, (fact_picking_lists.quantity  * multiply_hierarchy_unit_quantity(fact_picking_lists.stock_keeping_unit_id)) as liters
,dim_families.is_alcoholic as dim_families__is_alcoholic
,dim_picking_lists.owner_id as dim_picking_lists__owner_id
,dim_picking_lists.owner_type as dim_picking_lists__owner_type

from		fact_picking_lists
inner join	dim_dates on dim_dates.id = fact_picking_lists.date_id
inner join	dim_times on dim_times.id = fact_picking_lists.time_id
inner join	dim_families on dim_families.id = fact_picking_lists.family_id
inner join	dim_brands on dim_brands.id = dim_families.brand_id
inner join 	dim_picking_lists on dim_picking_lists.id = fact_picking_lists.picking_list_id
inner join 	dim_stock_keeping_units on dim_stock_keeping_units.id = fact_picking_lists.stock_keeping_unit_id
where fact_picking_lists.deleted_at is null
and dim_picking_lists.moderation_status = 1 -- Approved
and fact_picking_lists.direction = 'outbound'
order by  fact_picking_lists.id desc
'''

In [ ]:
# @title Define a SQL query to select data from a "fact_picking_lists" like in a View

sql = """\
      SELECT * FROM fact_picking_lists;
"""


# select
# fact_picking_lists.id as fact_picking_lists__id
# ,dim_dates."year" || '-' || dim_dates."month" || '-' || dim_dates."date" as fact_picking_lists__date
# ,to_char((LPAD((dim_times.hour_of_day || ''), 2, '0') || '' || LPAD((dim_times.minute_of_hour || ''), 2, '0') || '00')::time,'HH24:MI:SS AM') as fact_picking_lists__time
# ,GREATEST(
#     dim_picking_lists.created_at,
#     dim_picking_lists.updated_at,
#     dim_picking_lists.moderated_at, (
#     dim_picking_lists.document_date + '08:30:01'::time)
# ) as lasted_updated_at
# ,dim_picking_lists.document_number as dim_picking_lists__document_number
# ,dim_picking_lists.id as dim_picking_lists__id
# ,dim_stock_keeping_units.id as dim_stock_keeping_units__sku_id
# ,fact_picking_lists.quantity as fact_picking_lists__quantity
# -- , (fact_picking_lists.quantity  * multiply_hierarchy_unit_quantity(fact_picking_lists.stock_keeping_unit_id)) as liters
# ,dim_families.is_alcoholic as dim_families__is_alcoholic
# ,dim_picking_lists.owner_id as dim_picking_lists__owner_id
# ,dim_picking_lists.owner_type as dim_picking_lists__owner_type
# from		fact_picking_lists
# inner join	dim_dates on dim_dates.id = fact_picking_lists.date_id
# inner join	dim_times on dim_times.id = fact_picking_lists.time_id
# inner join	dim_families on dim_families.id = fact_picking_lists.family_id
# inner join	dim_brands on dim_brands.id = dim_families.brand_id
# inner join 	dim_picking_lists on dim_picking_lists.id = fact_picking_lists.picking_list_id
# inner join 	dim_stock_keeping_units on dim_stock_keeping_units.id = fact_picking_lists.stock_keeping_unit_id
# where fact_picking_lists.deleted_at is null
# and dim_picking_lists.moderation_status = 1 -- Approved
# and fact_picking_lists.direction = 'outbound'
# order by  fact_picking_lists.id desc

In [ ]:
sql

In [ ]:
# @title Create a cursor object to execute queries to dataframe

with conn.cursor() as cursor:
  # Execute the query
  cursor.execute("""
                  SELECT * FROM audits
                 """)
  columns = [desc[0] for desc in cursor.description]
  # Fetch all rows from the query result
  rows = cursor.fetchall()

  # Create dataframe
  df = pd.DataFrame(rows, columns=columns)
  # df.index =  df['fact_picking_lists__id']

  # # Close the cursor
  # cursor.close()

In [ ]:
df.head(5)

In [ ]:
# @title Save to csv for local work

# Convert the DataFrame to a dictionary with lists as values
data_dict = df.to_dict(orient="list")

# Specify the output file path
output_file = data_path/"output.csv"

# Write the query result to CSV
df.to_csv(output_file)

In [ ]:
# Close the connection
conn.close()

In [ ]:
df.head(10)

In [ ]:
df.describe()